# Function Calling in Semantic Kernel

This notebook demonstrates how functions work in the Microsoft Semantic Kernel. It shows how the model decides when to execute functions, how functions are invoked, and how Semantic Kernel orchestrates all the steps for function execution. You'll learn to define, register, and invoke both inline and C# plugin functions, and see how the AI model and kernel orchestration works to deliver response.

**Objectives:**
- Understand how Semantic Kernel orchestrates function execution.
- Define and use inline semantic functions.
- Pass parameters to functions.
- Register and invoke C# plugin (native) functions.


## Setup

In this section, we will set up the Semantic Kernel environment and configure it to use different LLM providers.

**Step 1:** Install NuGet packages

Install the required NuGet packages for Semantic Kernel and DotNetEnv to enable AI-powered workflows and environment variable management.

In [1]:
// Import Semantic Kernel
#r "nuget: Microsoft.SemanticKernel, 1.55.0"
#r "nuget: DotNetEnv, 3.1.0"

Installed Packages DotNetEnv, 3.1.0 Microsoft.SemanticKernel, 1.55.0

**Step 2:** Load environment variables

Load environment variables from a `.env` file if present. This helps manage secrets and configuration settings for your application.

In [2]:
using DotNetEnv;
using System.IO;

var envFilePath = Path.Combine(Environment.CurrentDirectory, "../..", ".env");
if (File.Exists(envFilePath))
{
    Env.Load(envFilePath);
    Console.WriteLine($"Loaded environment variables from {envFilePath}");
}
else
{
    Console.WriteLine($"No .env file found at {envFilePath}");
}

Loaded environment variables from d:\personal\aiagent-workshop\notebooks\semantic-kernel\../..\.env


**Step 3:** Instantiate the Semantic Kernel

Create and configure a Kernel instance, which will be used to interact with AI models.

In [3]:
using System.ClientModel;
using OpenAI;
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.ChatCompletion;
using System.Text;

OpenAIClient client = null;
if(Environment.GetEnvironmentVariable("USE_AZURE_OPENAI") == "true")
{
    // Configure Azure OpenAI client
    var azureEndpoint = Environment.GetEnvironmentVariable("AZURE_OPENAI_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("AZURE_OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(azureEndpoint) });
}
else if(Environment.GetEnvironmentVariable("USE_OPENAI") == "true")
{
    // Configure OpenAI client
    var apiKey = Environment.GetEnvironmentVariable("OPENAI_API_KEY");
    client = new OpenAIClient(new ApiKeyCredential(apiKey));
}
else if(Environment.GetEnvironmentVariable("USE_GITHUB") == "true")
{
    // Configure GitHub model client
    var uri = Environment.GetEnvironmentVariable("GITHUB_MODEL_ENDPOINT");
    var apiKey = Environment.GetEnvironmentVariable("GITHUB_TOKEN");
    client = new OpenAIClient(new ApiKeyCredential(apiKey), new OpenAIClientOptions { Endpoint = new Uri(uri) });
}

var modelId = Environment.GetEnvironmentVariable("MODEL");
// Create a chat completion service
var builder = Kernel.CreateBuilder();
builder.AddOpenAIChatCompletion(modelId, client);

// Get the chat completion service
Kernel kernel = builder.Build();

## Calling Functions (Semantic Functions)

This section demonstrates how to define and call functions within the Semantic Kernel. Functions can be defined as text (Semantic) or in C# (native). 

**Step 4:** Define an inline semantic function

Create an inline semantic function using a prompt template. This function will summarize input content, with a configurable word 
count.


Set up the execution settings for the inline function, such as maximum tokens, temperature, and top-p sampling. Then, create the function from the prompt template

In [4]:
string skPrompt = """
{{$input}}

Summarize the content above with a maximum of {{$wordCount}} words.

""";

var promptTemplateConfig = new PromptTemplateConfig(skPrompt);
var promptTemplateFactory = new KernelPromptTemplateFactory();
var promptTemplate = promptTemplateFactory.Create(promptTemplateConfig);
var summaryFunction = kernel.CreateFunctionFromPrompt(skPrompt);


**Step 6:** Prepare input content

Set up some content to summarize. In this example, the input describes AI agents and their capabilities.

In [6]:
var input = """
AI agents are autonomous software programs that use artificial intelligence to perform tasks, make decisions, and interact with their environment. They can process data, learn from experience, and adapt their behavior to achieve specific goals. AI agents are used in applications such as virtual assistants, robotics, automated trading, and customer service chatbots. Their ability to operate independently and improve over time makes them valuable tools in a wide range of industries.
""";

**Step 7:** Call the inline function

Send the input and word count to the inline function and receive a summary response from the AI model.

In [7]:
// Call the kernel to get a response 
var response = await kernel.InvokeAsync(summaryFunction, new() { ["input"] = input, ["wordCount"] = 10 });  
Console.WriteLine($"Response: {response}");

Response: AI agents autonomously perform tasks, learn, adapt, and improve efficiency.


## Calling Native Functions (C# Plugin Functions)
This section demonstrates how LLMs can automatically invoke C# plugin functions based on the input and the model's understanding of the task.

**Step 1:** Enable automatic function calling

You can configure the execution settings to allow the AI model to automatically call functions based on the prompt, similar to the function calling workflow.

In [8]:
using Microsoft.SemanticKernel;
using Microsoft.SemanticKernel.Connectors.OpenAI;

#pragma warning disable SKEXP0001
var openAiPromptSettings = new OpenAIPromptExecutionSettings
{
    FunctionChoiceBehavior = FunctionChoiceBehavior.Auto()
};
#pragma warning restore SKEXP0001

**Step 2:** Register a C# function and call it automatically

Define and register a C# function (plugin) with the kernel. Then, send a prompt that triggers the function call, and let the AI model invoke it automatically.

In [11]:
// Define a function that returns random weather
using System;
using Microsoft.SemanticKernel;

public class WeatherPlugin
{
    [KernelFunction]
    public string GetRandomWeather()
    {
        var weatherOptions = new[] { "Sunny", "Cloudy", "Rainy", "Windy", "Stormy", "Snowy" };
        var rnd = new Random();
        var temp = rnd.Next(10, 35);
        var weather = weatherOptions[rnd.Next(weatherOptions.Length)];
        return $"The weather is {weather} and {temp}°C.";
    }
}

// Register the plugin
var plugin = new WeatherPlugin();
kernel.ImportPluginFromObject(plugin, "weather");


Error: System.ArgumentException: An item with the same key has already been added. Key: weather
   at System.Collections.Generic.Dictionary`2.TryInsert(TKey key, TValue value, InsertionBehavior behavior)
   at System.Collections.Generic.Dictionary`2.Add(TKey key, TValue value)
   at Microsoft.SemanticKernel.KernelPluginCollection.Add(KernelPlugin plugin)
   at Microsoft.SemanticKernel.KernelExtensions.ImportPluginFromObject(Kernel kernel, Object target, String pluginName)
   at Submission#11.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationToken cancellationToken)

In [12]:
using Microsoft.SemanticKernel.Connectors.OpenAI;


var chat = kernel.GetRequiredService<IChatCompletionService>();
// Create a prompt that will trigger the function call
string prompt = "What is the weather today?";
var result = await chat.GetChatMessageContentAsync(prompt, openAiPromptSettings, kernel);
Console.WriteLine($"Response: {response}");


Response: AI agents autonomously perform tasks, learn, adapt, and improve efficiency.
